In [133]:
!ls -l /p/vast1/iopp/wisio/computed_llcs/genome/stage1

ls: cannot open directory /p/vast1/iopp/wisio/computed_llcs/genome/stage1: Permission denied


- STAGE2: Among the 5 dataframes  select dataframes which were derived from the same filtering (i.e., trange, proc_id, file_id and proc_id, trange, file_id)  (Hari on a notebook) (1 day)
- do a mean, median, and standard deviation of duration
- This test will tell us if order of columns matter for our dataset
- Expectation is mean, med, std are not gonna be close to the other dataset!!

- Access pattern : 0- seq and 1-ran
- io_cat: 0-other 1-read 2-write 3-metadata

In [134]:
import pandas as pd
import os

In [209]:
log_path = "/p/vast1/iopp/wisio/computed_llcs"
app_name = "montage"
index_val = 'file_id'
#folder = "_".join(index_name)
#print(folder)
#file = f"{log_path}/{app_name}/stage1/{folder}/part.0.parquet"

In [210]:
folder = f"{log_path}/{app_name}/stage1"
folder

'/p/vast1/iopp/wisio/computed_llcs/montage/stage1'

In [211]:
index_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
index_folders

['file_id',
 'file_id_proc_id',
 'file_id_proc_id_trange',
 'file_id_trange',
 'file_id_trange_proc_id',
 'proc_id',
 'proc_id_file_id',
 'proc_id_file_id_trange',
 'proc_id_trange',
 'proc_id_trange_file_id',
 'trange',
 'trange_file_id',
 'trange_file_id_proc_id',
 'trange_proc_id',
 'trange_proc_id_file_id']

In [212]:
df_map_general = {'trange':[], 'proc_id':[], 'file_id': []}
df_map_order = {'trange':[], 'proc_id':[], 'file_id': []}
for index in index_folders:
    values = index.split("_")
    index_list = []
    index_name_list = []
    trange_index = values.index('trange') if 'trange' in values else -1
    if trange_index != -1: 
        index_list.append(trange_index)
        index_name_list.append('trange')
    file_id_index = values.index('file') if 'file' in values else -1
    if file_id_index != -1: 
        index_list.append(file_id_index)
        index_name_list.append('file_id')
    proc_id_index = values.index('proc') if 'proc' in values else -1
    if proc_id_index != -1: 
        index_list.append(proc_id_index) 
        index_name_list.append('proc_id')
    indices = [x for _,x in sorted(zip(index_list,index_name_list))]
    #print(indices)
    file = f"{folder}/{index}/part.0.parquet"
    df = pd.read_parquet(file)
    temp_df = df.groupby([indices[-1]]).agg({'duration_sum':sum})
    temp_df = temp_df.rename(columns={"duration_sum": index})
    df_map_general[indices[-1]].append(temp_df) 
    if len(indices) == 3:
        df_map_order[indices[-1]].append(temp_df) 
        
    

In [213]:
outer_df_map ={}
for index, df_list in df_map_general.items():
    outer_df = pd.concat(df_list, axis=1) # outer_join
    outer_df_map[index] = outer_df
    outer_df_map[index]['mean'] = outer_df.mean(axis=1)
    outer_df_map[index]['std'] = outer_df.std(axis=1)
    outer_df_map[index]['std_per'] = outer_df_map[index]['std']/outer_df_map[index]['mean']*100
    outer_df_map[index] = outer_df_map[index].sort_values('std_per', ascending=False)

In [214]:
outer_df_map[index_val]

,file_id,proc_id_file_id,proc_id_trange_file_id,trange_file_id,trange_proc_id_file_id,mean,std,std_per
file_id,,,,,,,,
7668160539877627087,0.011279,0.009286,0.009286,0.011279,0.009286,0.010083,0.000976,9.679680
7662249566044993732,0.013529,0.012844,0.012844,0.013529,0.012844,0.013118,0.000335,2.556297
7649020242139592900,0.014611,0.013913,0.013913,0.014611,0.013913,0.014192,0.000342,2.410800
7641701892745115844,0.011022,0.010572,0.010572,0.011022,0.010572,0.010752,0.000221,2.052614
7668160540555917508,0.027627,0.026537,0.026537,0.027627,0.026537,0.026973,0.000534,1.980052
...,...,...,...,...,...,...,...,...
7652960893035267272,0.704955,0.704955,0.704955,0.704955,0.704955,0.704955,0.000000,0.000000
7652960892381028783,0.023258,0.023258,0.023258,0.023258,0.023258,0.023258,0.000000,0.000000
7652960890648958761,0.017807,0.017807,0.017807,0.017807,0.017807,0.017807,0.000000,0.000000


In [215]:
df_order_out_map ={}
for index, df_list in df_map_order.items():
    outer_df = pd.concat(df_list, axis=1)
    df_order_out_map[index] = outer_df
    df_order_out_map[index]['mean'] = outer_df.mean(axis=1)
    df_order_out_map[index]['std'] = outer_df.std(axis=1)
    df_order_out_map[index]['std_per'] = df_order_out_map[index]['std']/df_order_out_map[index]['mean'] * 100
    df_order_out_map[index] = df_order_out_map[index].sort_values('std_per', ascending=False)

In [216]:
df_order_out_map[index_val]

,proc_id_trange_file_id,trange_proc_id_file_id,mean,std,std_per
file_id,,,,,
300058772623729566,0.015766,0.015766,0.015766,0.0,0.0
7649301717116303556,0.013041,0.013041,0.013041,0.0,0.0
7645924016231192361,0.045970,0.045970,0.045970,0.0,0.0
7645924017395775684,0.013385,0.013385,0.013385,0.0,0.0
7645924018617500872,0.734405,0.734405,0.734405,0.0,0.0
...,...,...,...,...,...
5704378326418482595,0.070628,0.070628,0.070628,0.0,0.0
5704378327463932033,0.012254,0.012254,0.012254,0.0,0.0
5704378327514244910,0.015294,0.015294,0.015294,0.0,0.0


In [130]:
df.groupby([index_name[-1]]).agg({'duration_sum':sum})

,duration_sum
file_id,
4531734244631998940,0.013867
4531734244753862147,0.010060
4531734244818756329,0.024926
4531734244909082846,0.325141
4531734244977734007,0.357573
4531734245001276449,0.029139
4531734245058167199,1.985642
4531734245212906884,0.043319
4531734245256545024,0.197438
